# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "../output_data/cities.csv"
weather_df = weather_df = pd.read_csv(output_data_file)
weather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,taksimo,70,RU,1586122020,80,56.34,114.88,17.78,5.28
1,1,zhezkazgan,57,KZ,1586122020,62,47.80,67.71,47.53,7.45
2,2,bluff,100,NZ,1586122009,82,-46.60,168.33,60.01,4.00
3,3,yellowknife,20,CA,1586122017,45,62.46,-114.35,23.00,10.29
4,4,nikolskoye,1,RU,1586122020,74,59.70,30.79,28.40,4.47
...,...,...,...,...,...,...,...,...,...,...
595,595,ola,100,RU,1586122068,95,59.58,151.28,15.82,6.44
596,596,maunabo,90,PR,1586122068,69,18.01,-65.90,82.40,6.93
597,597,kyren,10,RU,1586122068,79,51.68,102.14,22.26,3.11
598,598,verkh-suetka,87,RU,1586122068,86,53.30,80.05,34.45,6.71


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
## Configure gmaps
gmaps.configure(api_key=g_key)
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity in hmuidty
humidity = weather_df["Humidity"]

# Plot Heatmap
figure_layout = {
    'width': '1200px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=20,
                                 point_radius=4)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Create ideal weather DataFrame
ideal_weather_df = weather_df.loc[(weather_df["Cloudiness"] == 0) & 
                                  (weather_df["Wind Speed"] < 10) & 
                                  (weather_df["Max Temp"] > 70) &
                                  (weather_df["Max Temp"] < 80)]

ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df                               


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
13,13,satrikh,0,IN,1586122020,50,26.87,81.20,75.20,3.36
178,178,ormara,0,PK,1586122034,59,25.21,64.64,78.46,4.92
207,207,sombrio,0,BR,1586122037,75,-29.11,-49.62,71.20,9.78
254,254,veraval,0,IN,1586122040,62,20.90,70.37,77.18,9.10
263,263,bela,0,IN,1586122041,27,25.93,81.98,77.11,5.48
417,417,hualmay,0,PE,1586122053,63,-11.10,-77.61,77.65,8.66
423,423,beitbridge,0,ZW,1586122054,37,-22.22,30.00,71.87,3.24
452,452,mopipi,0,BW,1586122056,28,-21.18,24.88,74.52,0.60
454,454,badarpur,0,IN,1586122056,48,24.90,92.60,70.36,2.15
473,473,silvassa,0,IN,1586122058,63,20.27,73.02,74.82,2.21


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_weather_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
13,13,satrikh,0,IN,1586122020,50,26.87,81.20,75.20,3.36,
178,178,ormara,0,PK,1586122034,59,25.21,64.64,78.46,4.92,
207,207,sombrio,0,BR,1586122037,75,-29.11,-49.62,71.20,9.78,
254,254,veraval,0,IN,1586122040,62,20.90,70.37,77.18,9.10,
263,263,bela,0,IN,1586122041,27,25.93,81.98,77.11,5.48,
417,417,hualmay,0,PE,1586122053,63,-11.10,-77.61,77.65,8.66,
423,423,beitbridge,0,ZW,1586122054,37,-22.22,30.00,71.87,3.24,
452,452,mopipi,0,BW,1586122056,28,-21.18,24.88,74.52,0.60,
454,454,badarpur,0,IN,1586122056,48,24.90,92.60,70.36,2.15,
473,473,silvassa,0,IN,1586122058,63,20.27,73.02,74.82,2.21,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map